HNSW Algorithm


In [ ]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

###Data Charactersitics: https://archive.ics.uci.edu/ml/datasets/SIFT10M

In [ ]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [ ]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [ ]:
!ls


sample_data  sift  sift.tar.gz


In [ ]:
# data we will search through
wb = read_fvecs('/content/sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [ ]:
print (xq)

[[  1.   3.  11. 110.  62.  22.   4.   0.  43.  21.  22.  18.   6.  28.
   64.   9.  11.   1.   0.   0.   1.  40. 101.  21.  20.   2.   4.   2.
    2.   9.  18.  35.   1.   1.   7.  25. 108. 116.  63.   2.   0.   0.
   11.  74.  40. 101. 116.   3.  33.   1.   1.  11.  14.  18. 116. 116.
   68.  12.   5.   4.   2.   2.   9. 102.  17.   3.  10.  18.   8.  15.
   67.  63.  15.   0.  14. 116.  80.   0.   2.  22.  96.  37.  28.  88.
   43.   1.   4.  18. 116.  51.   5.  11.  32.  14.   8.  23.  44.  17.
   12.   9.   0.   0.  19.  37.  85.  18.  16. 104.  22.   6.   2.  26.
   12.  58.  67.  82.  25.  12.   2.   2.  25.  18.   8.   2.  19.  42.
   48.  11.]]


In [ ]:
xq.shape

(1, 128)

In [ ]:
wb.shape

(1000000, 128)

In [ ]:
# 1M samples
xb = read_fvecs('/content/sift/sift_base.fvecs')
# queries
xq = read_fvecs('/content/sift/sift_base.fvecs')[0].reshape(1, -1)
xq_full = read_fvecs('/content/sift/sift_query.fvecs')

In [ ]:
!pip install faiss

     |████████████████████████████████| 4.7 MB 9.8 MB/s 


In [ ]:
!sudo apt-get install libopenblas-dev
!sudo apt-get install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-dev is already the newest version (0.2.20+ds-4).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (236 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot

In [ ]:
import faiss

In [ ]:
# setup our HNSW parameters
d = 128  # vector size
M = 32
efSearch = 32  # number of entry points (neighbors) we use on each layer
efConstruction = 32  # number of entry points used on each layer
                     # during construction

index = faiss.IndexHNSWFlat(d, M)
print(index.hnsw)

<faiss.swigfaiss.HNSW; proxy of <Swig Object of type 'faiss::HNSW *' at 0x7fcf0a2c0ea0> >


In [ ]:
# the HNSW index starts with no levels
index.hnsw.max_level

-1

In [ ]:
# and levels (or layers) are empty too
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([], dtype=int64)

We can set the efConstruction and efSearch parameters, only efConstruction must be set before building the index. efSearch only affects search time behavior.

In [ ]:
index.hnsw.efConstruction = efConstruction
index.hnsw.efSearch = efSearch

In [ ]:
index.add(xb)

In [ ]:
# after adding our data we will find that the level
# has been set automatically
index.hnsw.max_level

4

In [ ]:
# and levels (or layers) are now populated
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([     0, 968746,  30276,    951,     26,      1])

In [ ]:
index.hnsw.entry_point


118295

The HNSW::set_default_probas function (from HNSW.cpp)calculates the number of neighbors (in total) a vertex will have across the calculated number of layers. We find that Faiss' implementation does not use M_max or M_max0 directly, but instead uses M to set these values. M_max is set to M, and M_max is set to 2*M.

In [ ]:
def set_default_probas(M: int, m_L: float):
    nn = 0  # set nearest neighbors count = 0
    cum_nneighbor_per_level = []
    level = 0  # we start at level 0
    assign_probas = []
    while True:
        # calculate probability for current level
        proba = np.exp(-level / m_L) * (1 - np.exp(-1 / m_L))
        # once we reach low prob threshold, we've created enough levels
        if proba < 1e-9: break
        assign_probas.append(proba)
        # neighbors is == M on every level except level 0 where == M*2
        nn += M*2 if level == 0 else M
        cum_nneighbor_per_level.append(nn)
        level += 1
    return assign_probas, cum_nneighbor_per_level

In [ ]:
assign_probas, cum_nneighbor_per_level = set_default_probas(
    32, 1/np.log(32)
)
assign_probas, cum_nneighbor_per_level

([0.96875,
  0.030273437499999986,
  0.0009460449218749991,
  2.956390380859371e-05,
  9.23871994018553e-07,
  2.887099981307982e-08],
 [64, 96, 128, 160, 192, 224])